In [1]:
from mysql import connector
import pandas as pd
import numpy as np

print(pd.__version__)
print(np.__version__)
print("설치 완료")

2.2.3
2.2.5
설치 완료


## DB 연결 코드
- 여기서부터 시작

In [2]:
import os
from mysql import connector

PASSWORD = 'sebin1234'

try:
    with connector.connect(
        host = "localhost",
        user = "root",
        password = PASSWORD
    ) as database: 
        print(f"Database object: {database}")
except connector.Error as e: 
    print(e)

Database object: <mysql.connector.connection_cext.CMySQLConnection object at 0x000001EC89C9DA90>


# DB 연결 추천 방식

In [12]:
from dotenv import load_dotenv
import os
import mysql.connector
from mysql.connector import pooling

# .env 파일 로드
load_dotenv()

# 데이터베이스 연결 설정
db_config = {
    'host': os.getenv('MYSQL_HOST'),
    'port': int(os.getenv('MYSQL_PORT')),
    'user': os.getenv('MYSQL_USER'),
    'password': os.getenv('MYSQL_PASSWORD'),
    'database': os.getenv('MYSQL_DATABASE'),
    'charset': os.getenv('MYSQL_CHARSET')
}

# 커넥션 풀 설정
pool_config = {
    'pool_name': os.getenv('MYSQL_POOL_NAME'),
    'pool_size': int(os.getenv('MYSQL_POOL_SIZE')),
    'pool_reset_session': True,
    **db_config
}

# 커넥션 풀 생성
connection_pool = mysql.connector.pooling.MySQLConnectionPool(**pool_config)

# 커넥션 풀 사용 예시
def get_connection():
    return connection_pool.get_connection()

# 사용 예시
try:
    connection = get_connection()
    cursor = connection.cursor()

    sql_query = '''
        SELECT 1+1;
    '''
    
    # 쿼리 실행
    cursor.execute("SELECT 1+1;")
    results = cursor.fetchall()
    
    for row in results:
        print(row)
        
except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    if 'cursor' in locals():
        cursor.close()
    if 'connection' in locals():
        connection.close()

(2,)


# 스키마 생성

In [24]:
import os
from mysql import connector

PASSWORD = 'sebin1234'

try:
    with connector.connect(
        host = "localhost",
        user = "root",
        password = PASSWORD
    ) as database: 
        print(f"Database object: {database}")
        delete_db = 'DROP DATABASE book_ratings;'             # 스키마 삭제 명령어
        create_db = 'CREATE DATABASE book_ratings;'           # 스키마 생성 명령어
        with database.cursor() as cursor:
            # cursor.execute(delete_db)
            cursor.execute(create_db)

            # 현재 존재하는 스키마 목록 확인
            show_db = 'SHOW DATABASES;'
            cursor.execute(show_db)
            for db in cursor:
                print(db)
except connector.Error as e: 
    print(e)

Database object: <mysql.connector.connection_cext.CMySQLConnection object at 0x000001EC89D1EFD0>
('book_ratings',)
('classicmodels',)
('datasets',)
('information_schema',)
('instacart',)
('mssqldb',)
('mysql',)
('performance_schema',)
('sakila',)
('sys',)
('testdb',)
('titanic',)
('world',)


In [19]:
# 특정 스키마 지정 후, 연결
import os
from mysql import connector

PASSWORD = 'sebin1234'

try:
    with connector.connect(
        host = "localhost",
        user = "root",
        password = PASSWORD,
        database = 'book_ratings' # 데이터베이스 스키마 지정
    ) as database: 
        print(f"Database object: {database}")
except connector.Error as e: 
    print(e)

Database object: <mysql.connector.connection_cext.CMySQLConnection object at 0x000001ECE897E850>


In [25]:
# 특정 스키마 지정 후, 연결 
import os
from mysql import connector

create_book_table = """
CREATE TABLE books(
    id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(100),
    author VARCHAR(100),
    genre VARCHAR(100),
    release_year YEAR(4)
)
"""

try:
    with connector.connect(
        host = "localhost",
        user = "root",
        password = 'sebin1234', 
        database = 'book_ratings' # 차이점
    ) as database: 
        print(f"Database object: {database}")
        with database.cursor() as cursor:
            cursor.execute(create_book_table)
            database.commit()

            # 테이블 확인 
            describe_books = 'DESCRIBE books;'
            cursor.execute(describe_books)
            result = cursor.fetchall()   # 모든 테이블 정보를 다 가지고 오는데, 
                                         # pandas 데이터프레임으로 오는 것이 아니라 리스트-튜플형태로 저장
            for col in result:
                print(col)
            
except connector.Error as e: 
    print(e)

Database object: <mysql.connector.connection_cext.CMySQLConnection object at 0x000001ECE897E850>
('id', 'int', 'NO', 'PRI', None, 'auto_increment')
('title', 'varchar(100)', 'YES', '', None, '')
('author', 'varchar(100)', 'YES', '', None, '')
('genre', 'varchar(100)', 'YES', '', None, '')
('release_year', 'year', 'YES', '', None, '')


In [28]:
# INSERT SINGLE RECORD
insert_single_record = "INSERT INTO books (id, title, author, genre, release_year)\
    VALUES (%s, %s, %s, %s, %s)"
single_record = (
    "1", "Choose Yourself! Be Happy, Make Millions, Live the Dream", "James Altucher", "self-help", "2013"
    )

try: 
    # Connect to existing database
    with connector.connect(
        host = "localhost",
        user = "root",
        password = PASSWORD,
        database = "book_ratings"
    ) as existing_database:
        
        # Create cursor object
        with existing_database.cursor() as cursor:
            cursor.execute(insert_single_record, single_record)
            existing_database.commit()
        
except connector.Error as e: 
    print(e)

# INSERT MULTIPLE RECORDS
insert_multiple_records = "INSERT INTO books (id, title, author, genre, release_year)\
    VALUES (%s, %s, %s, %s, %s)"
multiple_records = [
    (
        "2", 
        "Skip the Line: The 10,000 Experiments Rule and Other Surprising Advice for Reaching Your Goals",
        "James Altucher",
        "self-help",
        "2021"        
    ),
    (
        "3",
        "The Power of No: Because One Little Word Can Bring Health, Abundance, and Happiness",
        "James Altucher",
        "self-help",
        "2014"
    ),
    (
        "4",
        "The 48 Laws of Power",
        "Robert Greene",
        "self-help",
        "1998"
    ),
    (
        "5",
        "Mastery",
        "Robert Greene",
        "self-help",
        "2012"
    ),
    (
        "6",
        "The Art of Seduction",
        "Robert Greene",
        "self-help",
        "2001"
    ),
]

try: 
    # Connect to existing database
    with connector.connect(
        host = "localhost",
        user = "root",
        password = PASSWORD,
        database = "book_ratings"
    ) as existing_database:
        
        # Create cursor object
        with existing_database.cursor() as cursor:
            cursor.executemany(insert_multiple_records, multiple_records)
            existing_database.commit()
        
except connector.Error as e: 
    print(e)

1062 (23000): Duplicate entry '1' for key 'books.PRIMARY'
1062 (23000): Duplicate entry '2' for key 'books.PRIMARY'


# 데이터 조회

In [29]:
import os
from mysql import connector
import pandas as pd  # 추가

PASSWORD = 'sebin1234'

try:
    with connector.connect(
        host = "localhost",
        user = "root",
        password = PASSWORD,
        database = 'book_ratings'
    ) as database: 
        print(f"Database object: {database}")

        # 조회 쿼리
        select_query = 'SELECT * FROM books;'

        with database.cursor() as cursor:
            cursor.execute(select_query)

            # fetchall()로 데이터 가져오기
            rows = cursor.fetchall()

            # cursor.description을 통해 컬럼명 가져오기
            column_names = [desc[0] for desc in cursor.description]

            # pandas DataFrame으로 변환
            df = pd.DataFrame(rows, columns=column_names)

            # 출력
            print(df)

except connector.Error as e: 
    print(e)

Database object: <mysql.connector.connection_cext.CMySQLConnection object at 0x000001ECE897E850>
('James Altucher', 2013)
('James Altucher', 2021)
('James Altucher', 2014)
('Robert Greene', 1998)
('Robert Greene', 2012)
('Robert Greene', 2001)


In [ ]:
# 미션 : 어떤 형태로든 Table을 조회해도 Table의 고유한 컬럼 & 데이터프레임으로 만들기

In [30]:
import os
from mysql import connector
import pandas as pd  # 추가

PASSWORD = 'sebin1234'

try:
    with connector.connect(
        host = "localhost",
        user = "root",
        password = PASSWORD,
        database = 'book_ratings'
    ) as database: 
        print(f"Database object: {database}")

        # 조회 쿼리
        select_query = 'SELECT * FROM books;'

        with database.cursor() as cursor:
            cursor.execute(select_query)

            # fetchall()로 데이터 가져오기
            rows = cursor.fetchall()

            # cursor.description을 통해 컬럼명 가져오기
            column_names = [desc[0] for desc in cursor.description]

            # pandas DataFrame으로 변환
            df = pd.DataFrame(rows, columns=column_names)

            # 출력
            print(df)

except connector.Error as e: 
    print(e)

Database object: <mysql.connector.connection_cext.CMySQLConnection object at 0x000001EC89D1EFD0>
   id                                              title          author  \
0   1  Choose Yourself! Be Happy, Make Millions, Live...  James Altucher   
1   2  Skip the Line: The 10,000 Experiments Rule and...  James Altucher   
2   3  The Power of No: Because One Little Word Can B...  James Altucher   
3   4                               The 48 Laws of Power   Robert Greene   
4   5                                            Mastery   Robert Greene   
5   6                               The Art of Seduction   Robert Greene   

       genre  release_year  
0  self-help          2013  
1  self-help          2021  
2  self-help          2014  
3  self-help          1998  
4  self-help          2012  
5  self-help          2001  


In [32]:
# 미션: 어떤 형태로든 Table을 조회해도 고유한 컬럼명을 가진 DataFrame으로 반환하는 함수 작성

import mysql.connector as connector  # MySQL 데이터베이스 연결을 위한 라이브러리
import pandas as pd  # 데이터를 DataFrame 형태로 다루기 위한 pandas

PASSWORD = 'sebin1234'  # 실제 비밀번호로 대체 필요

# 데이터베이스에서 쿼리 결과를 가져와서 pandas DataFrame으로 반환하는 함수
def fetch_data_to_dataframe(query, database=None):
    try: 
        # 기존 데이터베이스에 연결
        with connector.connect(
            host="localhost",       # 호스트 주소
            user="root",            # 사용자명
            password=PASSWORD,      # 비밀번호
            database=database       # 사용할 데이터베이스 이름
        ) as existing_database:
            
            # 커서(cursor) 객체 생성 (쿼리 실행에 사용)
            with existing_database.cursor() as cursor:
                cursor.execute(query)  # SQL 쿼리 실행
                
                # 컬럼 이름 추출 (cursor.description에서 컬럼명만 가져오기)
                column_names = [i[0] for i in cursor.description]
                
                # 모든 결과 데이터 가져오기
                returned_data = cursor.fetchall()
                
                # pandas DataFrame으로 변환
                df = pd.DataFrame(returned_data, columns=column_names)
                print(df)
                #return df  # DataFrame 반환
    
    except connector.Error as e: 
        # 예외 발생 시 에러 출력 및 None 반환
        print(e)
        return None

select_query = 'SELECT * FROM books;'
df_specific_cols = fetch_data_to_dataframe(select_query, "book_ratings")

   id                                              title          author  \
0   1  Choose Yourself! Be Happy, Make Millions, Live...  James Altucher   
1   2  Skip the Line: The 10,000 Experiments Rule and...  James Altucher   
2   3  The Power of No: Because One Little Word Can B...  James Altucher   
3   4                               The 48 Laws of Power   Robert Greene   
4   5                                            Mastery   Robert Greene   
5   6                               The Art of Seduction   Robert Greene   

       genre  release_year  
0  self-help          2013  
1  self-help          2021  
2  self-help          2014  
3  self-help          1998  
4  self-help          2012  
5  self-help          2001  


In [34]:
# 미션: 어떤 형태로든 Table을 조회해도 고유한 컬럼명을 가진 DataFrame으로 반환하는 함수 작성

import mysql.connector as connector  # MySQL 데이터베이스 연결을 위한 라이브러리
import pandas as pd  # 데이터를 DataFrame 형태로 다루기 위한 pandas

PASSWORD = 'sebin1234'  # 실제 비밀번호로 대체 필요

# 데이터베이스에서 쿼리 결과를 가져와서 pandas DataFrame으로 반환하는 함수
def fetch_data_to_dataframe(query, database=None):
    try: 
        # 기존 데이터베이스에 연결
        with connector.connect(
            host="localhost",       # 호스트 주소
            user="root",            # 사용자명
            password=PASSWORD,      # 비밀번호
            database=database       # 사용할 데이터베이스 이름
        ) as existing_database:
            
            # 커서(cursor) 객체 생성 (쿼리 실행에 사용)
            with existing_database.cursor() as cursor:
                cursor.execute(query)  # SQL 쿼리 실행
                
                # 컬럼 이름 추출 (cursor.description에서 컬럼명만 가져오기)
                column_names = [i[0] for i in cursor.description]
                
                # 모든 결과 데이터 가져오기
                returned_data = cursor.fetchall()
                
                # pandas DataFrame으로 변환
                df = pd.DataFrame(returned_data, columns=column_names)
                return df  # DataFrame 반환
    
    except connector.Error as e: 
        # 예외 발생 시 에러 출력 및 None 반환
        print(e)
        return None

# [예제 1] 특정 컬럼(author, release_year)만 조회
query_specific_cols = "SELECT author, release_year FROM books"
df_specific_cols = fetch_data_to_dataframe(query_specific_cols, "book_ratings")
if df_specific_cols is not None:
    print("특정 컬럼을 선택한 DataFrame:")
    print(df_specific_cols)

# [예제 2] 전체 컬럼(*) 조회
query_all_cols = "SELECT * FROM books"
df_all_cols = fetch_data_to_dataframe(query_all_cols, "book_ratings")
if df_all_cols is not None:
    print("모든 컬럼을 포함한 DataFrame:")
    print(df_all_cols)

특정 컬럼을 선택한 DataFrame:
           author  release_year
0  James Altucher          2013
1  James Altucher          2021
2  James Altucher          2014
3   Robert Greene          1998
4   Robert Greene          2012
5   Robert Greene          2001
모든 컬럼을 포함한 DataFrame:
   id                                              title          author  \
0   1  Choose Yourself! Be Happy, Make Millions, Live...  James Altucher   
1   2  Skip the Line: The 10,000 Experiments Rule and...  James Altucher   
2   3  The Power of No: Because One Little Word Can B...  James Altucher   
3   4                               The 48 Laws of Power   Robert Greene   
4   5                                            Mastery   Robert Greene   
5   6                               The Art of Seduction   Robert Greene   

       genre  release_year  
0  self-help          2013  
1  self-help          2021  
2  self-help          2014  
3  self-help          1998  
4  self-help          2012  
5  self-help          200